## Setup the tracing
To start tracing your RAG you can add LangWatch wrappers to your functions. This is an example of tracing of your application in production. You can read more about it here - [RAGs Context Tracking](https://docs.langwatch.ai/rags/rags-context-tracking).

In [2]:
import langwatch 
from langwatch import openai
from openai import OpenAI
import dotenv
import os

dotenv.load_dotenv()

openai_api_key = os.getenv("OPENAI_API_KEY")

client = OpenAI(api_key=openai_api_key)

with openai.OpenAITracer(client):
  with langwatch.capture_rag(
      input="What is the capital of France?",
      contexts=[
          {
              "document_id": "doc-1",
              "chunk_id": "0",
              "content": "France is a country in Europe.",
          },
          {
              "document_id": "doc-2",
              "chunk_id": "0",
              "content": "Paris is the capital of France.",
          },
      ],
  ):
      response = client.chat.completions.create(
          model="gpt-3.5-turbo",
          messages=[
              {"role": "user", "content": "What is the capital of France?"}
          ],
      )


print(response.choices[0].message)


ChatCompletionMessage(content='The capital of France is Paris.', role='assistant', function_call=None, tool_calls=None)


/Users/zhenyabudnyk/DevProjects/langwatch-saas/langevals/notebooks/.venv/lib/python3.12/site-packages/langwatch/tracer.py:260: UserWarning: LANGWATCH_API_KEY is not set, LLMs traces will not be sent, go to https://langwatch.ai to set it up
  warn(


## Evaluate
If you want to evaluate a few simple examples that you created manually or copy pasted from the dataset - you can do it in the following way.
Import the evaluators of your choice from `langevals` module. In case of RAG evaluations you need only `Entry` and `Evaluator` objects. Then, you can fill in the arguments of `Entry` instances with `input` - question to the RAG app, `output` - the generated response and `contexts` - retrieved contexts that were used for generation. For evaluation to run successfully you need at least two input parameters.
Finally, you can run each evaluator and get the evaluation `Result` objects.  

In [ ]:
%pip install "langevals[all]"

In [14]:
from langevals_ragas.context_relevancy import RagasContextRelevancyEntry, RagasContextRelevancyEvaluator
from langevals_ragas.faithfulness import RagasFaithfulnessEntry, RagasFaithfulnessEvaluator
from langevals_ragas.answer_relevancy import RagasAnswerRelevancyEntry, RagasAnswerRelevancyEvaluator


entry1 = RagasAnswerRelevancyEntry(input="What is the capital of France?", output="Paris is the capital of France")

entry2 = RagasContextRelevancyEntry(output="Paris is the capital of France", contexts=[
    "Water can evaporate or turn into ice",
    "Dogs and Cats can be friends",
    "The sun is shining today"
])

entry3 = RagasFaithfulnessEntry(output="Paris is the capital of France", contexts=[
    "France is a country in Europe",
    "Lyon, Paris and Bordeaux are cities in France",
    "Paris is the capital of France"
])

result1 = RagasAnswerRelevancyEvaluator().evaluate(entry=entry1)
result2 = RagasContextRelevancyEvaluator().evaluate(entry=entry2)
result3 = RagasFaithfulnessEvaluator().evaluate(entry=entry3)
print(result1)
print(result2)
print(result3)


status='processed' score=1.0 passed=None details=None cost=Money(currency='USD', amount=0.002036)
status='processed' score=0.3333333333333333 passed=None details=None cost=Money(currency='USD', amount=0.00033600000000000004)
status='processed' score=1.0 passed=None details=None cost=Money(currency='USD', amount=0.0038910000000000004)
